In [1]:
import random
from PIL import Image, ImageDraw, ImageFilter
import os
from random import randint
import pandas as pd
import numpy as np

In [2]:
if not os.path.isdir("dataset"): #Создаем основной репозиторий
    os.mkdir("dataset")
os.chdir("dataset") # Указываем используемый репозиторий

# Настраиваемые гиперпараметры
pack = 5 # Количество папок с изображениями
n_step = 5 # Количество изображений в папке
pix_x = 250 # Размер изображения в пикселях
pix_y = 250
debug = False # При значении True размер и количество клеток устанавливаются вручную
sizeR = 3 #Размер точек
sizeG = 5
delta = 5 # Шаг изменения
red_min = 100 # Минимальное количество эритроцитов на изображении
red_max = 500 # Максимальное количество эритроцитов на изображении
gray_min = 5 # Минимальное количество лимфоцитов на изображении
gray_max = 25 # Максимальное количество лимфоцитов на изображении
color_f = 'white' # Цвет фона изображения
format_image = '.png' # Формат сохраняемого изображения
name_image = 'broun' # Имя изображения
name_files = "traffic" #Имя папки
color_ery = 'red' # Цвет эритроцитов
color_limf = 'grey' # Цвет лимфоцитов
address = "/content/dataset" # Расположение репозитория
is_blurry = False # Включение размытия
blur_radius = 0.5
is_noisy = True # Включение шума
noise_cutoff = 1 # Если случайное значение с нормальным распределением будет выше данного значения - на изображении появится "плохой" пиксель, если ниже - без изменений
# Для noise_cutoff = 1 вероятность пояявления шума составляет 0.1587
noise_prob = 0.5 # Вероятность добавления шума: необходима для уменьшения вероятности появления шума. При значении 1 появление шума зависит только от noise_cutoff
noise_color = "random" # Цвет шума: grey или random

ran_coord_rx = [] #Создание используемых массивов
ran_coord_ry = []
ran_coord_gx = []
ran_coord_gy = []
Wxr = []
Wyr = []
Wxg = []
Wyg = []
files = []
erythrocytes = []
lymphocytes = []
noise = int(is_noisy)
blur = int(is_blurry)

for h in range(1, pack+1): # Цикл для создания папок в основном репозитории
    hh = str(h)
    if not os.path.isdir(name_files+hh): #Создание папки
        os.mkdir(name_files+hh)
    os.chdir(name_files+hh) # Название папки
    rep = name_files+hh
    print(os.getcwd()) # Проверка репозитория
    if debug:
      ran_red = randint(red_min, red_max) #Количество точек в одной папке
      ran_gray = randint(gray_min, gray_max)
    else:
      sizeR = round(np.random.normal(loc=4))
      sizeG = round(np.random.normal(loc=5))
      ran_red = round(np.random.normal(loc=4)*100)
      ran_gray = round(np.random.normal(loc=3)*10)
    files.append(rep)
    erythrocytes.append(ran_red)
    lymphocytes.append(ran_gray)
    for i in range(0,n_step):

        img = Image.new('RGBA', (pix_x, pix_y), color_f) # Создание изображения
        idraw = ImageDraw.Draw(img) # Создание объекта изображения
        Wxr.clear() # Обнуление списков
        Wyr.clear()
        Wxg.clear()
        Wyg.clear()
        if i == 0: # Генерация начального положения эритроцитов на изображении
            for m in range(0, ran_red):
                ran_coord_rx.append(float(random.uniform(0, pix_x)))
                ran_coord_ry.append(float(random.uniform(0, pix_y)))
        print('ran_coord_rx', ran_coord_rx)
        for a in range(0, ran_red): # Цикл для отображения эритроцитов
            ran_coord_rx.append(float(random.uniform((ran_coord_rx[0] - delta), (ran_coord_rx[0] + delta)))) # Определение следующей координаты
            ran_coord_ry.append(float(random.uniform((ran_coord_ry[0] - delta), (ran_coord_ry[0] + delta))))
            ran_coord_rx.remove(ran_coord_rx[0])
            ran_coord_ry.remove(ran_coord_ry[0])
            yi = np.random.choice([1, -1])
            Wxr.append(np.ones(n_step) * ran_coord_rx[0])
            Wyr.append(np.ones(n_step) * ran_coord_ry[0])
            Wxr[0] = Wxr[a-1] + (yi / np.sqrt(n_step)) # Уравнение броуновского движения
            Wyr[0] = Wyr[a - 1] + (yi / np.sqrt(n_step))
            wxr1 = Wxr[0]
            wxr = wxr1[0]
            wyr1 = Wyr[0]
            wyr = wyr1[0]
            xr1 = wxr - sizeR
            yr1 = wyr - sizeR
            xr2 = wxr + sizeR
            yr2 = wyr + sizeR
            idraw.ellipse((xr1, yr1, xr2, yr2), color_ery) # Отображение частицы
            istr = str(i)
            img.save(name_image + istr + format_image) # Сохраннение изображения

        if i == 0: # Генерация начального положения лейкоцитов на изображении
            for z in range(0, ran_gray):
                ran_coord_gx.append(float(randint(0, pix_x)))
                ran_coord_gy.append(float(randint(0, pix_y)))
        print('ran_coord_gx',ran_coord_gx)
        for p in range(0, ran_gray): # Цикл для отображения лейкоцитов
            ran_coord_gx.append(float(random.uniform((ran_coord_gx[0] - delta), (ran_coord_gx[0] + delta)))) # Определение следующей координаты
            ran_coord_gy.append(float(random.uniform((ran_coord_gy[0] - delta), (ran_coord_gy[0] + delta))))
            ran_coord_gx.remove(ran_coord_gx[0])
            ran_coord_gy.remove(ran_coord_gy[0])
            yi = np.random.choice([1, -1])
            Wxg.append(np.ones(n_step) * ran_coord_gx[p])
            Wyg.append(np.ones(n_step) * ran_coord_gy[p])
            Wxg[0] = Wxg[p - 1] + (yi / np.sqrt(n_step)) # Уравнение броуновского движения
            Wyg[0] = Wyg[p - 1] + (yi / np.sqrt(n_step))
            wxg1 = Wxg[0]
            wxg = wxg1[0]
            wyg1 = Wyg[0]
            wyg = wyg1[0]
            xg1 = wxg - sizeG
            yg1 = wyg - sizeG
            xg2 = wxg + sizeG
            yg2 = wyg + sizeG
            idraw.ellipse((xg1, yg1, xg2, yg2), color_limf) # Отображение частицы
            istr = str(i)
            img.save(name_image + istr + format_image) # Сохраннение изображения

        if is_blurry: # При включении размытия применяется Box Blur с радиусом blur_radius
            img = img.filter(ImageFilter.BoxBlur(blur_radius))
            idraw = ImageDraw.Draw(img)
            istr = str(i)
            img.save(name_image + istr + format_image)

        if is_noisy: # При включении шумов
            output = np.random.normal(size=(pix_x, pix_y))
            output[output < noise_cutoff] = 0 # Данные строки создают матрицу размером pix_x и pix_y, состоящую из 0 и 1, где 0 - "нормальный" пиксель, 1 - вероятно "плохой" пиксель
            output[output > noise_cutoff] = 1
            for x in range(pix_x):
              for y in range(pix_y):
                if output[x][y] == 1:
                  if random.uniform(0, 1) < noise_prob: # Если случайное число в диапазоне от 0 до 1 окажется меньше, чем noise_prob, то текущий пиксель является "плохим" (появляется шум)
                    if noise_color == "grey":
                      color_part = random.randint(0, 255)
                      color = (color_part, color_part, color_part) # Цифровой шум серого цвета разной интенсивности
                    elif noise_color == "random":
                      color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) # Цифровой шум случайного цвета
                    idraw.rectangle(((x, y), (x, y)), color) # Отображение шума
            istr = str(i)
            img.save(name_image + istr + format_image)

    os.chdir(address) # Возвращение к исходному репозиторию
os.chdir("/content")
frame = pd.DataFrame( {'files' : files, 'erythrocytes' : erythrocytes, 'lymphocytes' : lymphocytes, 'blur' : blur, 'noise' : noise})
print(frame)
frame.to_csv("./DataFrame.csv")

/content/dataset/traffic1
ran_coord_rx [201.80893231800158, 109.72193310424355, 170.86324930251382, 203.8446447446081, 171.66344640253, 180.09965717493247, 96.92669028143975, 10.425664679989094, 14.828621504537814, 43.0225774445869, 95.03187543188422, 245.80802821658335, 76.09824684737909, 118.76809695683282, 112.19772721538229, 221.00370505108714, 182.9968492510801, 105.06445559852834, 181.07057755783947, 76.00564518896091, 13.062403908859627, 63.663622006747886, 117.6283351628721, 124.1523933492746, 48.88757893731488, 247.04670875219136, 4.152309417012362, 215.62071561546267, 192.51844883107927, 34.020590463684854, 73.4949478394108, 215.2943892636004, 114.67528643519795, 110.43078599421763, 177.61341001245512, 212.24688028205568, 212.23307334878754, 2.2970628447027686, 194.63164858038112, 89.5506651958016, 86.19138986944758, 157.00985382868646, 171.5561422544812, 198.50009868927418, 83.7214524165881, 88.99038190002251, 188.13246042636845, 77.99719766514951, 215.87510785828727, 119.78